In [ ]:
import os
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir(os.pardir)

import scipy.spatial as ss
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

from src.utils.dataloader import load_ear_data
from src.utils.viz import set_axes_equal

In [ ]:
df = load_ear_data('te', 26, 1e-3)

In [ ]:
x = df['x [mm]'].to_numpy()
y = df['y [mm]'].to_numpy()
z = df['z [mm]'].to_numpy()
xyz = np.c_[x, y, z]

In [ ]:
df

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection ='3d')
ax.plot(x, y, z, '.', ms=1, alpha=1, c='k')
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]')
ax = set_axes_equal(ax)
ax.view_init(30, 50)

fig.tight_layout();